In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Union

In [8]:
def generate_sample_donations(filename: str = 'donations.txt', num_entries: int = 50) -> None:
    """
    Generate realistic donation data with controlled randomness
    
    Args:
        filename (str): Output file for donation data
        num_entries (int): Number of donation entries to generate
    """
    # Set random seed for reproducibility
    np.random.seed(42)
    
    # Generate donation amounts using normal distribution
    # Mean: $250, Standard Deviation: $100
    # Constrain values between $50 and $1000
    donations = np.clip(
        np.random.normal(250, 100, num_entries), 
        50, 
        1000
    )
    
    # Round to nearest dollar
    donations = np.round(donations, 2)
    
    # Write to file
    try:
        with open(filename, 'w') as f:
            for donation in donations:
                f.write(f"{donation}\n")
        print(f"Generated {num_entries} donations in {filename}")
    except IOError as e:
        print(f"Error writing file: {e}")

In [9]:
def read_donations(filename: str) -> pd.Series:
    """
    Read donations from a text file
    
    Parameters:
    -----------
    filename : str
        Path to the donations file
    
    Returns:
    --------
    pd.Series
        Validated donation amounts
    """
    # TODO: Implement robust file reading
    # Requirements:
    # - Handle potential file reading errors
    # - Convert to numeric, handling potential non-numeric entries
    # - Remove or handle outliers
    pass

In [10]:
def calculate_statistics(donations: pd.Series) -> dict:
    """
    Calculate comprehensive statistics
    
    Parameters:
    -----------
    donations : pd.Series
        Donation amounts
    
    Returns:
    --------
    dict
        Comprehensive statistical summary
    """
    return {
        'Mean': donations.mean(),
        'Median': donations.median(),
        'Mode': donations.mode().values[0],
        'Standard Deviation': donations.std(),
        'Minimum': donations.min(),
        'Maximum': donations.max(),
        'Total Donations': donations.sum()
    }

In [11]:
def create_donation_visualizations(donations: pd.Series) -> None:
    """
    Create multiple visualizations of donation data
    
    Parameters:
    -----------
    donations : pd.Series
        Donation amounts
    """
    # Create a 2x2 subplot grid
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Histogram
    donations.hist(ax=axes[0, 0], bins=20)
    axes[0, 0].set_title('Donation Distribution')
    axes[0, 0].set_xlabel('Donation Amount ($)')
    axes[0, 0].set_ylabel('Frequency')
    
    # Box Plot
    donations.plot.box(ax=axes[0, 1])
    axes[0, 1].set_title('Donation Box Plot')
    
    # Violin Plot
    donations.plot.violin(ax=axes[1, 0])
    axes[1, 0].set_title('Donation Violin Plot')
    
    # Cumulative Distribution
    donations.cumsum().plot(ax=axes[1, 1])
    axes[1, 1].set_title('Cumulative Donation Amount')
    axes[1, 1].set_xlabel('Number of Donations')
    axes[1, 1].set_ylabel('Total Donation Amount ($)')
    
    plt.tight_layout()
    plt.show()

In [12]:
def main():
    # Generate sample data
    generate_sample_donations()
    
    # Read donations
    donations = read_donations('donations.txt')
    
    # Calculate statistics
    stats = calculate_statistics(donations)
    print("Donation Statistics:")
    for stat, value in stats.items():
        print(f"{stat}: ${value:,.2f}")
    
    # Create visualizations
    create_donation_visualizations(donations)

# Run the analysis
if __name__ == '__main__':
    main()

Generated 50 donations in donations.txt


AttributeError: 'NoneType' object has no attribute 'mean'